# accelerator

## LOGON

In [9]:
from amocrm.v2 import tokens
from amocrm.v2.exceptions import UnAuthorizedException, AmoApiException

In [10]:
from accelerator_config import keys
tokens.default_token_manager(
    storage=tokens.FileTokensStorage(directory_path="./test_dir/accelerator"),  # by default FileTokensStorage
)

In [11]:
tokens.default_token_manager.init(
    code='def50200be2abfea2417b7031a536afaab20963e1b10d432661d3f78d18b16b319ffa6c13604f09db3169bdda060f52c2ee7560724add60f013de0afb3945872af3f291705f921e04480183b16c2dd0a724f06c6db27f7ff4e6751c29cec9cebe8cccd9900ae1c8751a0697eb41335753a98297a7e2ac555e3884a9b1cdafda9188310260dc305e1929a73322378c1536c8dabb2b05b6795d6b8f56df97f2e01443b2fc843c996b4c41ee1a6f753d68cae2a1ef9bd6a611f0806f7aeb7ed71088925d7d160b4c55077ea50c4b2186d72f186eea8de23837a37a0cc6ed8028f79516040656dcd870ba9164c954ae7f4ddbcf4ab0454ae2f296733ddeebb86b126eaec67d71847a5eac0b3caeb3fa41c85cc3fce4312bb6a933a8521552bbf604a55c40367da7361a11380b9d370b28c2cccb5b3924042e130c3f2d98b1a18d7edc2f03ca4ba850e710fab0031a4b945d220aa2f693fd2e58e998e811c0141566b03130609ce0736b1edd81dbc6b88805c2c468ded91a52ba8cca60482c66f18898a385725d0b215227cd77584151c1bdabc5b8ff54a82c8bc56fa5dc11b2c0c14cc14c08bb17241ec350b615f89de9aaad03c3a66f49b0f089074ef33635cc36faa6fb613cdabc39945d50e625d9fa72af48d9c34bc79503ad4ef7313',
    skip_error=False
)

## Pipelines

In [12]:
from amocrm.v2 import Pipeline, Status

In [13]:
pipelines = list(Pipeline.objects.all())

In [14]:
pipelines_dicts = []
for pipeline in pipelines:
    pipelines_dicts.append(pipeline.__dict__['_data'])

In [15]:
import pandas as pd

In [16]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [17]:
df_pipelines = pd.DataFrame(pipelines_dicts)

df_pipelines.drop(["_links", "_embedded"], axis=1, inplace=True)

In [18]:
from google.cloud import bigquery
client = bigquery.Client()

In [19]:
df_pipelines.to_gbq(
    "accelerator_oddjob.dw_amocrm_alc_pipelines",
    if_exists="replace"
)

## Users

In [20]:
from amocrm.v2 import User

In [21]:
users = list(User.objects.all())

In [22]:
users_dicts = []
for user in users:
    users_dicts.append(user.__dict__['_data'])

In [23]:
for user in users_dicts:
    user['is_admin'] = users_dicts[0]['rights']['is_admin']
    user['is_active'] = users_dicts[0]['rights']['is_active']
    user['group_id'] = users_dicts[0]['rights']['group_id']
    user['role_id'] = users_dicts[0]['rights']['role_id']

In [24]:
df_users = pd.DataFrame(users_dicts)
df_users.drop(["rights", "_links"], axis=1, inplace=True)

In [25]:
df_users.to_gbq(
    "accelerator_oddjob.dw_amocrm_alc_users", if_exists="replace"
)

## Status names

In [26]:
from amocrm.v2 import Status, filters
from amocrm.v2.exceptions import AmoApiException

In [27]:
df_statuses = pd.DataFrame.from_dict({
    'id': ['']
})

In [29]:
df_temp_pipe = pd.read_gbq(
    """
    SELECT DISTINCT
        id
    FROM
        `oddjob-db-2007.franchise_oddjob.dw_amocrm_fr_pipelines`
    """
)
pipeline_ids = df_temp_pipe['id'].to_list()
statuses = []
for pipeline in pipeline_ids:
    df_temp_stat = pd.read_gbq(
        f"""
        SELECT DISTINCT
            id_status_before
        FROM
            `oddjob-db-2007.accelerator_oddjob.dw_amocrm_acl_events`
        WHERE id_pipeline_before = {pipeline}
        """
    )
    status_ids = df_temp_stat["id_status_before"].to_list()
    for status in status_ids:
        try:
            statuses.append(
                Status.get_for(f"{pipeline}").get(f"{status}").__dict__["_data"]
            )
        except AmoApiException:
            pass

GenericGBQException: Reason: 404 Not found: Table oddjob-db-2007:accelerator_oddjob.dw_amocrm_acl_events was not found in location US

Location: US
Job ID: b8a792eb-a4d1-4c91-be0b-6106d65cbff5


# partner

## LOGON

In [30]:
from amocrm.v2 import tokens
from amocrm.v2.exceptions import UnAuthorizedException, AmoApiException

In [31]:
from partner_config import keys
tokens.default_token_manager(
    storage=tokens.FileTokensStorage(directory_path="./test_dir/partner"),  # by default FileTokensStorage
)

In [32]:
tokens.default_token_manager.init(
    code='def50200ebee8796d00c61a949a005f81f721a56327f1f5d89e9353b0b31b0834a0fd6436e23d6f2b00391feea89f1de4dfef512ad6251b973cdf68590d0079bd25662bc4944551423b1ba377840051d0fabe6a5c0296c6d859a3cde49b56d069db57d0d1abe1805b39e9f930afea114d0d6ff33da0b2947b1c1892aec3bdcde15606332440965f4637fe95b55d1f9629bfb07991b9c4e8a26ac8307df8bdd6f073bb58f80a0b2e773c6c1b29ec93b352274aae31ad35a5f7dfb5571f0c67610843a8e4e1f3c5cfce1e6b114870ca78bf492948d099e0db94540a71bea10767c003058eedcaaf4abf5a57c9ac2878d5e444b2933d974f6d97e9a252d73424ff05b043c492b50ccf9712baf4ae5e345a305b64ae172e01ed88b6e526c53adc06375a4145f701d178b27ed038a3e8b880af584872f262f33e28d40810d4991a7cb5aa76a09e5ce825ed4cb247666292fb16b7207e644158afc8815116a6a652d6cc38e6d4b5e2343c25fbcd0acef323c01e06e876db8f887cf268a87775bae6fa7292c44cf51eaa112a58912e39b85c019d1fcaa1fc1bb6d15e0061fbfd3eb6ede364d3fa55f10b7b3a1d487dd422fbbc92737bd0079b89decef020423862016c6874b27274556f4773a16195cc88877f0ec5c9c1b01dc79fcddc062a17ccb7f94ed',
    skip_error=False
)

## Pipelines

In [33]:
from amocrm.v2 import Pipeline, Status

In [34]:
pipelines = list(Pipeline.objects.all())

In [35]:
pipelines_dicts = []
for pipeline in pipelines:
    pipelines_dicts.append(pipeline.__dict__['_data'])

In [36]:
import pandas as pd

In [37]:
df_pipelines = pd.DataFrame(pipelines_dicts)

In [38]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [39]:
df_pipelines = pd.DataFrame(pipelines_dicts)

df_pipelines.drop(["_links", "_embedded"], axis=1, inplace=True)

In [40]:
from google.cloud import bigquery
client = bigquery.Client()

In [41]:
df_pipelines.to_gbq(
    "partner.dw_amocrm_ptr_pipelines",
    if_exists="replace"
)

## Users

In [43]:
from amocrm.v2 import User

In [44]:
users = list(User.objects.all())

In [45]:
users_dicts = []
for user in users:
    users_dicts.append(user.__dict__['_data'])

In [46]:
for user in users_dicts:
    user['is_admin'] = users_dicts[0]['rights']['is_admin']
    user['is_active'] = users_dicts[0]['rights']['is_active']
    user['group_id'] = users_dicts[0]['rights']['group_id']
    user['role_id'] = users_dicts[0]['rights']['role_id']

In [47]:
df_users = pd.DataFrame(users_dicts)
df_users.drop(["rights", "_links"], axis=1, inplace=True)

In [48]:
df_users.to_gbq(
    "partner_oddjob.dw_amocrm_ptr_users", if_exists="replace"
)